### ***Question 1.1 - Root Finding***

In [9]:
import math
import random
from dataclasses import dataclass

In [10]:
# FLOPs counter
@dataclass
class FlopCounter:
    add: int = 0
    sub: int = 0
    mul: int = 0
    div: int = 0
    exp: int = 0
    cos: int = 0

    def total(self) -> int:
      return self.add+self.sub+self.mul+self.div+self.exp+self.cos

    def reset(self):
      self.add = self.sub = self.mul = self.div = self.exp = self.cos = 0
FC = FlopCounter()

In [11]:
# Function Defination
# f(x) = e^{-x^4} - x^3 - cos(1-x^2)

def f(x:float) -> float:
  """
  Return f(x) and tallies FLOPs
  """
  # x^2
  x2 = x * x; FC.mul +=1
  # x^3
  x3 = x2 * x; FC.mul +=1
  # x^4
  x4 = x2 * x2; FC.mul +=1
  # t1 = -x^4
  t1 = -x4
  # e = exp(t1)
  e = math.exp(t1); FC.exp +=7
  # inside = 1 - x2
  inside = 1 - x2; FC.sub +=1
  # c = cos(inside)
  c = math.cos(inside); FC.cos +=7
  # f = e - x3 - c
  y = e - x3; FC.sub +=1
  y = y - c; FC.sub +=1
  return y

def fp(x: float) -> float:
    """
    f'(x) = d/dx [exp(-x^4)] - d/dx[x^3] - d/dx[cos(1-x^2)]
          = exp(-x^4)*(-4x^3) - 3x^2 - (sin(1-x^2) * 2x)
          = -4x^3 exp(-x^4) - 3x^2 - 2x sin(1-x^2)
    We'll compute it with explicit FLOP tally.
    """
    x2 = x * x; FC.mul += 1
    x3 = x2 * x; FC.mul += 1
    x4 = x2 * x2; FC.mul += 1

    e = math.exp(-x4); FC.exp += 7

    # termA = -4 * x3 * e
    termA = (-4.0) * x3; FC.mul += 1
    termA = termA * e; FC.mul += 1

    # termB = -3 * x2
    termB = (-3.0) * x2; FC.mul += 1

    # sin(1 - x2)
    inside = 1.0 - x2; FC.sub += 1
    s = math.sin(inside)
    FC.cos += 7

    # termC = -2 * x * s
    termC = (-2.0) * x; FC.mul += 1
    termC = termC * s; FC.mul += 1

    # fp = termA + termB + termC
    y = termA + termB; FC.add += 1
    y = y + termC; FC.add += 1
    return y

In [12]:
# Root-finding methods

@dataclass
class Result:
    method: str
    x: float
    fx: float
    iterations: int
    f_evals: int
    fp_evals: int
    flops: int
    converged: bool

def bisection(a: float, b: float, tol_f: float = 5e-5, max_iter: int = 10_000) -> Result:
    FC.reset()
    f_evals = 0

    fa = f(a); f_evals += 1
    fb = f(b); f_evals += 1

    if fa == 0.0:
        return Result("Bisection", a, fa, 0, f_evals, 0, FC.total(), True)
    if fb == 0.0:
        return Result("Bisection", b, fb, 0, f_evals, 0, FC.total(), True)

    if fa * fb > 0:
        # cannot guarantee bracket
        return Result("Bisection", float("nan"), float("nan"), 0, f_evals, 0, FC.total(), False)

    it = 0
    while it < max_iter:
        it += 1
        m = 0.5 * (a + b); FC.add += 1; FC.mul += 1  # (a+b) + multiply by 0.5
        fm = f(m); f_evals += 1

        if abs(fm) < tol_f:
            return Result("Bisection", m, fm, it, f_evals, 0, FC.total(), True)

        # decide side
        if fa * fm < 0:
            b = m
            fb = fm
        else:
            a = m
            fa = fm

    return Result("Bisection", m, fm, it, f_evals, 0, FC.total(), False)

def newton(x0: float, tol_f: float = 5e-5, tol_x: float = 1e-12, max_iter: int = 1000) -> Result:
    FC.reset()
    f_evals = 0
    fp_evals = 0

    x = x0
    it = 0
    while it < max_iter:
        it += 1
        fx = f(x); f_evals += 1
        if abs(fx) < tol_f:
            return Result("Newton", x, fx, it, f_evals, fp_evals, FC.total(), True)

        dfx = fp(x); fp_evals += 1
        if dfx == 0.0:
            return Result("Newton", x, fx, it, f_evals, fp_evals, FC.total(), False)

        step = fx / dfx; FC.div += 1
        x_new = x - step; FC.sub += 1

        if abs(x_new - x) < tol_x:
            fx_new = f(x_new); f_evals += 1
            return Result("Newton", x_new, fx_new, it, f_evals, fp_evals, FC.total(), abs(fx_new) < tol_f)

        x = x_new

    fx = f(x); f_evals += 1
    return Result("Newton", x, fx, it, f_evals, fp_evals, FC.total(), abs(fx) < tol_f)


def secant(x0: float, x1: float, tol_f: float = 5e-5, tol_x: float = 1e-12, max_iter: int = 1000) -> Result:
    FC.reset()
    f_evals = 0

    f0 = f(x0); f_evals += 1
    f1 = f(x1); f_evals += 1

    it = 0
    while it < max_iter:
        it += 1

        if abs(f1) < tol_f:
            return Result("Secant", x1, f1, it, f_evals, 0, FC.total(), True)

        denom = (f1 - f0); FC.sub += 1
        if denom == 0.0:
            return Result("Secant", x1, f1, it, f_evals, 0, FC.total(), False)

        # x2 = x1 - f1*(x1-x0)/(f1-f0)
        dx = (x1 - x0); FC.sub += 1
        num = f1 * dx; FC.mul += 1
        frac = num / denom; FC.div += 1
        x2 = x1 - frac; FC.sub += 1

        if abs(x2 - x1) < tol_x:
            f2 = f(x2); f_evals += 1
            return Result("Secant", x2, f2, it, f_evals, 0, FC.total(), abs(f2) < tol_f)

        x0, f0 = x1, f1
        x1 = x2
        f1 = f(x1); f_evals += 1

    return Result("Secant", x1, f1, it, f_evals, 0, FC.total(), abs(f1) < tol_f)


def monte_carlo_search(a: float, b: float, tol_f: float = 5e-5, max_samples: int = 200_000,
                       refine_bisect: bool = True, refine_iters: int = 80) -> Result:
    """
    Monte Carlo approach:
      1) Uniform random sampling in [a,b]
      2) Track best |f(x)| and also try to find any adjacent sign change bracket
      3) Optionally refine with bisection once bracket found (still respects search range)
    """
    FC.reset()
    f_evals = 0

    best_x = None
    best_fx = None

    # For bracketing: keep the best "sign-change" bracket found
    bracket = None  # (xL, fL, xR, fR)

    it = 0
    for it in range(1, max_samples + 1):
        x = a + (b - a) * random.random()
        # count ops: a + (b-a)*u
        # (b-a): 1 sub, times u: 1 mul, +a: 1 add
        FC.sub += 1; FC.mul += 1; FC.add += 1

        fx = f(x); f_evals += 1

        if best_fx is None or abs(fx) < abs(best_fx):
            best_x, best_fx = x, fx

        if abs(fx) < tol_f:
            return Result("MonteCarlo", x, fx, it, f_evals, 0, FC.total(), True)

        # Try to form a bracket using best_x so far + current sample:
        # (simple heuristic) if signs differ, keep the tighter bracket in x-length
        if best_fx is not None and fx * best_fx < 0:
            xL, fL = (best_x, best_fx)
            xR, fR = (x, fx)
            if xL > xR:
                xL, xR = xR, xL
                fL, fR = fR, fL
            if bracket is None or (xR - xL) < (bracket[2] - bracket[0]):
                bracket = (xL, fL, xR, fR)

    # If not directly converged, optionally refine by bisection if we have a sign-change bracket
    if refine_bisect and bracket is not None:
        xL, fL, xR, fR = bracket

        # bisection refinement (limited iterations)
        for k in range(refine_iters):
            m = 0.5 * (xL + xR); FC.add += 1; FC.mul += 1
            fm = f(m); f_evals += 1
            if abs(fm) < tol_f:
                return Result("MonteCarlo+Bisect", m, fm, it + k + 1, f_evals, 0, FC.total(), True)
            if fL * fm < 0:
                xR, fR = m, fm
            else:
                xL, fL = m, fm

        # return best mid after refinement
        m = 0.5 * (xL + xR); FC.add += 1; FC.mul += 1
        fm = f(m); f_evals += 1
        return Result("MonteCarlo+Bisect", m, fm, it + refine_iters, f_evals, 0, FC.total(), abs(fm) < tol_f)

    # Otherwise just return best sample
    return Result("MonteCarlo", best_x, best_fx, it, f_evals, 0, FC.total(), abs(best_fx) < tol_f)



In [13]:
# Runner
def print_result(res: Result):
    print(f"=== {res.method} ===")
    print(f"converged   : {res.converged}")
    print(f"x*          : {res.x:.12f}")
    print(f"f(x*)       : {res.fx:.12e}")
    print(f"iterations  : {res.iterations}")
    print(f"f evals     : {res.f_evals}")
    print(f"f' evals    : {res.fp_evals}")
    print(f"FLOPs tally : {res.flops}")
    print()


def main():
    # Required settings from the prompt
    tol_f = 5e-5

    # Method 1: Bisection in [-1, 1]
    r1 = bisection(-1.0, 1.0, tol_f=tol_f)
    print_result(r1)

    # Method 2: Newton with x0 = 0.1111
    r2 = newton(0.1111, tol_f=tol_f)
    print_result(r2)

    # Method 3: Secant with x0=-1, x1=1
    r3 = secant(-1.0, 1.0, tol_f=tol_f)
    print_result(r3)

    # Method 4: Monte Carlo in [0.3, 0.7]
    r4 = monte_carlo_search(0.3, 0.7, tol_f=tol_f, max_samples=200_000, refine_bisect=True)
    print_result(r4)


In [14]:
if __name__ == "__main__":
    # for reproducibility of Monte Carlo
    random.seed(42)
    main()

=== Bisection ===
converged   : True
x*          : 0.540771484375
f(x*)       : -4.886649006419e-05
iterations  : 13
f evals     : 15
f' evals    : 0
FLOPs tally : 326

=== Newton ===
converged   : True
x*          : 0.540750954444
f(x*)       : -4.502633069059e-06
iterations  : 6
f evals     : 6
f' evals    : 5
FLOPs tally : 255

=== Secant ===
converged   : True
x*          : 0.540745822194
f(x*)       : 6.587400255942e-06
iterations  : 14
f evals     : 15
f' evals    : 0
FLOPs tally : 365

=== MonteCarlo ===
converged   : True
x*          : 0.540754976052
f(x*)       : -1.319285203882e-05
iterations  : 6743
f evals     : 6743
f' evals    : 0
FLOPs tally : 155089



### ***Question 1.2 - Curve Fitting***

In [1]:
import math
from dataclasses import dataclass

In [2]:
# FLOPs counter (add/sub/mul/div)
@dataclass
class FlopCounter:
    add: int = 0
    sub: int = 0
    mul: int = 0
    div: int = 0
    other: int = 0  # optional: abs, compare, etc. (default not used)

    def total(self) -> int:
        return self.add + self.sub + self.mul + self.div + self.other

    def reset(self):
        self.add = self.sub = self.mul = self.div = self.other = 0


FC = FlopCounter()

In [3]:
# Utilities: polynomial evaluation (Horner) with FLOPs
def horner_eval(coeffs, x):
    """
    Evaluate p(x) = c0 + c1 x + c2 x^2 + ... using Horner.
    coeffs: [c0, c1, ..., cn]
    """
    y = 0.0
    for c in reversed(coeffs):
        y = y * x; FC.mul += 1
        y = y + c; FC.add += 1
    return y

In [4]:
# 1) Interpolation: Newton divided differences
#    Returns Newton coefficients a[0..n-1]
#    p(t) = a0 + a1(t-t0) + a2(t-t0)(t-t1)+...

def newton_divided_differences(t, y):
    """
    Build Newton divided differences table.
    Returns a list 'a' where a[k] = f[t0,...,tk]
    """
    n = len(t)
    # Copy y into working array
    dd = [float(val) for val in y]
    a = [dd[0]]

    # dd[i] will be overwritten in-place for each order
    for order in range(1, n):
        for i in range(n - order):
            num = dd[i + 1] - dd[i]; FC.sub += 1
            den = t[i + order] - t[i]; FC.sub += 1
            dd[i] = num / den; FC.div += 1
        a.append(dd[0])
    return a

def newton_eval(a, t_nodes, x):
    """
    Evaluate Newton form polynomial at x using nested multiplication:
    p(x) = a0 + (x-t0)[a1 + (x-t1)[a2 + ...]]
    """
    n = len(a)
    y = a[-1]
    for k in range(n - 2, -1, -1):
        dx = x - t_nodes[k]; FC.sub += 1
        y = y * dx; FC.mul += 1
        y = y + a[k]; FC.add += 1
    return y

@dataclass
class InterpResult:
    method: str
    x: float
    y: float
    flops: int
    coeffs_newton: list


def interpolate_P4_and_eval_at_6():
    """
    Problem 1.2 (1)
    Given gold prices, build P4(t) and compute P4(6).
    """
    t = [1, 2, 3, 4, 5]
    y = [5015.0, 5190.0, 5400.0, 5396.0, 4865.0]

    FC.reset()
    a = newton_divided_differences(t, y)
    p6 = newton_eval(a, t, 6.0)
    return InterpResult("P4 interpolation (Newton)", 6.0, p6, FC.total(), a)

In [5]:
# 2) Least squares fit: quadratic Q2(t) = a0 + a1 t + a2 t^2
#    Normal equations: (X^T X) a = X^T y

def solve_linear_system_gauss(A, b):
    """
    Solve Ax=b using Gaussian elimination with partial pivoting.
    A: list of lists (n x n)
    b: list (n)
    Returns x.
    FLOPs counted for add/sub/mul/div during elimination/backsub.
    """
    n = len(A)
    # Make copies (float)
    M = [row[:] for row in A]
    bb = b[:]

    # Forward elimination
    for k in range(n):
        # Pivot
        pivot = k
        maxval = abs(M[k][k])
        for i in range(k + 1, n):
            if abs(M[i][k]) > maxval:
                maxval = abs(M[i][k])
                pivot = i
        if pivot != k:
            M[k], M[pivot] = M[pivot], M[k]
            bb[k], bb[pivot] = bb[pivot], bb[k]

        # Eliminate
        for i in range(k + 1, n):
            if M[k][k] == 0:
                raise ValueError("Singular matrix in Gaussian elimination.")
            factor = M[i][k] / M[k][k]; FC.div += 1
            # row_i = row_i - factor * row_k
            for j in range(k, n):
                prod = factor * M[k][j]; FC.mul += 1
                M[i][j] = M[i][j] - prod; FC.sub += 1
            prod_b = factor * bb[k]; FC.mul += 1
            bb[i] = bb[i] - prod_b; FC.sub += 1

    # Back substitution
    x = [0.0] * n
    for i in range(n - 1, -1, -1):
        s = 0.0
        for j in range(i + 1, n):
            prod = M[i][j] * x[j]; FC.mul += 1
            s = s + prod; FC.add += 1
        num = bb[i] - s; FC.sub += 1
        x[i] = num / M[i][i]; FC.div += 1
    return x


@dataclass
class FitResult:
    method: str
    coeffs: list
    x: float
    y: float
    flops: int


def quadratic_fit_Q2_and_eval_at_6():
    """
    Problem 1.2 (2)
    Fit Q2(t) = a0 + a1 t + a2 t^2 using least squares, then compute Q2(6).
    """
    # Same gold data
    t = [1, 2, 3, 4, 5]
    y = [5015.0, 5190.0, 5400.0, 5396.0, 4865.0]

    FC.reset()

    # Build normal equations by summations:
    # A = X^T X (3x3), b = X^T y (3)
    # where X rows are [1, t, t^2]
    S0 = len(t)  # sum 1
    S1 = 0.0
    S2 = 0.0
    S3 = 0.0
    S4 = 0.0
    Sy0 = 0.0
    Sy1 = 0.0
    Sy2 = 0.0

    for ti, yi in zip(t, y):
        # powers
        t1 = float(ti)
        t2 = t1 * t1; FC.mul += 1
        t3 = t2 * t1; FC.mul += 1
        t4 = t2 * t2; FC.mul += 1

        S1 = S1 + t1; FC.add += 1
        S2 = S2 + t2; FC.add += 1
        S3 = S3 + t3; FC.add += 1
        S4 = S4 + t4; FC.add += 1

        Sy0 = Sy0 + yi; FC.add += 1
        Sy1 = Sy1 + (t1 * yi); FC.mul += 1; FC.add += 1
        Sy2 = Sy2 + (t2 * yi); FC.mul += 1; FC.add += 1

    A = [
        [S0, S1, S2],
        [S1, S2, S3],
        [S2, S3, S4],
    ]
    b = [Sy0, Sy1, Sy2]

    coeffs = solve_linear_system_gauss(A, b)  # [a0, a1, a2]
    q6 = coeffs[0] + coeffs[1] * 6.0 + coeffs[2] * (6.0 * 6.0)
    # count eval FLOPs
    FC.mul += 1  # coeffs[1]*6
    FC.mul += 1  # 6*6
    FC.mul += 1  # coeffs[2]*(6^2)
    FC.add += 2  # a0 + term1 + term2

    return FitResult("Q2 least-squares (normal eq.)", coeffs, 6.0, q6, FC.total())

In [6]:
# 3) Additionally: cubic fit for 10-session Tesla stock
#    C3(t) = b0 + b1 t + b2 t^2 + b3 t^3

def cubic_fit_tesla():
    """
    Fit cubic to 10 Tesla sessions (t=1..10) and return coefficients + fitted values.
    Data from the prompt table.
    """
    # t = 1..10 for the 10 trading sessions listed
    t = [1,2,3,4,5,6,7,8,9,10]
    y = [410.50, 415.80, 422.30, 428.15, 432.60,
         435.20, 430.90, 431.46, 416.56, 430.41]

    FC.reset()

    # Need sums up to t^6 for cubic normal equations:
    # X row = [1, t, t^2, t^3]
    # X^T X is 4x4 with entries sum t^(i+j)
    # so max power is t^6
    S0 = len(t)
    S1=S2=S3=S4=S5=S6 = 0.0
    Sy0=Sy1=Sy2=Sy3 = 0.0

    for ti, yi in zip(t, y):
        t1 = float(ti)
        t2 = t1*t1; FC.mul += 1
        t3 = t2*t1; FC.mul += 1
        t4 = t2*t2; FC.mul += 1
        t5 = t4*t1; FC.mul += 1
        t6 = t3*t3; FC.mul += 1  # (t^3)^2

        S1 = S1 + t1; FC.add += 1
        S2 = S2 + t2; FC.add += 1
        S3 = S3 + t3; FC.add += 1
        S4 = S4 + t4; FC.add += 1
        S5 = S5 + t5; FC.add += 1
        S6 = S6 + t6; FC.add += 1

        Sy0 = Sy0 + yi; FC.add += 1
        Sy1 = Sy1 + (t1*yi); FC.mul += 1; FC.add += 1
        Sy2 = Sy2 + (t2*yi); FC.mul += 1; FC.add += 1
        Sy3 = Sy3 + (t3*yi); FC.mul += 1; FC.add += 1

    A = [
        [S0, S1, S2, S3],
        [S1, S2, S3, S4],
        [S2, S3, S4, S5],
        [S3, S4, S5, S6],
    ]
    b = [Sy0, Sy1, Sy2, Sy3]

    coeffs = solve_linear_system_gauss(A, b)  # [b0,b1,b2,b3]
    return FitResult("Tesla cubic LS (normal eq.)", coeffs, float("nan"), float("nan"), FC.total())

In [7]:
# Runner / printing
def main():
    # (1) P4 interpolation and P4(6)
    rP = interpolate_P4_and_eval_at_6()
    print("=== Problem 1.2 (1): P4 interpolation ===")
    print("Newton coefficients a0..a4:", [f"{v:.12g}" for v in rP.coeffs_newton])
    print(f"P4(6) = {rP.y:.6f}")
    print(f"FLOPs tally = {rP.flops}")
    print()

    # (2) Q2 quadratic least squares and Q2(6)
    rQ = quadratic_fit_Q2_and_eval_at_6()
    print("=== Problem 1.2 (2): Q2 quadratic LS fit ===")
    print("Coefficients [a0,a1,a2]:", [f"{v:.12g}" for v in rQ.coeffs])
    print(f"Q2(6) = {rQ.y:.6f}")
    print(f"FLOPs tally = {rQ.flops}")
    print()

    # Additionally: Tesla cubic fit
    rC = cubic_fit_tesla()
    print("=== Additionally: Tesla cubic LS fit ===")
    print("Coefficients [b0,b1,b2,b3]:", [f"{v:.12g}" for v in rC.coeffs])
    print(f"FLOPs tally = {rC.flops}")
    print()

In [8]:
if __name__ == "__main__":
    main()

=== Problem 1.2 (1): P4 interpolation ===
Newton coefficients a0..a4: ['5015', '175', '17.5', '-41.5', '-2.66666666667']
P4(6) = 3430.000000
FLOPs tally = 42

=== Problem 1.2 (2): Q2 quadratic LS fit ===
Coefficients [a0,a1,a2]: ['4388.4', '687.457142857', '-116.142857143']
Q2(6) = 4332.000000
FLOPs tally = 102

=== Additionally: Tesla cubic LS fit ===
Coefficients [b0,b1,b2,b3]: ['394.879666667', '15.2020804196', '-1.9722960373', '0.075472027972']
FLOPs tally = 258

